In [4]:
import pandas as pd
import numpy as np
import joblib
import sklearn
# import ensemble
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,mean_squared_error,balanced_accuracy_score,recall_score
import random
import lightgbm as lgb
import catboost as cbt
import xgboost as xgb

In [5]:
def incr_act_top10(input_df,pred_col,cm_key='customer',treated_col='ind_recommended',actual_col='activation'):
    
	#for correcting variable types
    input_df[[treated_col, actual_col, pred_col]] = input_df[[treated_col, actual_col, pred_col]].apply(pd.to_numeric, errors='coerce')
	
    input_df['rank_per_cm1'] = input_df.groupby(cm_key)[pred_col].rank(method='first', ascending=False)
    
    input_df = input_df.loc[input_df.rank_per_cm1 <= 10,:]
    
    agg_df = input_df.groupby(treated_col,as_index=False).agg({actual_col:'mean'})
    agg_df.columns = [treated_col,'avg_30d_act']
    
    print(agg_df)
    recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col]==1,'avg_30d_act'])
    not_recommended_avg_30d_act = float(agg_df.loc[agg_df[treated_col]==0,'avg_30d_act'])
    
    return (recommended_avg_30d_act-not_recommended_avg_30d_act)

In [6]:
def evalu(df_input, df_round): 
    # round off scores to 10 decimal points
    df_input['predicted_score'] = df_input['predicted_score'].round(10)


    # groupby customer, merchant and max score
    df_input = df_input.groupby(['customer', 'merchant'], as_index = False)['predicted_score'].agg('max')


    # merging predicted file and dependent variable file
    eval_data = pd.merge(df_round,df_input,on=['customer','merchant'],how='inner').drop_duplicates()
    # deleting the rows having null value in predicted_score
    eval_data = eval_data[~(eval_data['predicted_score'].isna())]


    if df_round.shape[0] != eval_data.shape[0]:
        return('Error: Rows are missing in the output file')
    else:
        print('Input Files are Correct')

    final_score = round(incr_act_top10(input_df=eval_data,pred_col='predicted_score',cm_key='customer',treated_col='ind_recommended',actual_col='activation'), 7)

    print('Incremental Activation Rate for Top 10 ranked Merchants(dataset level): ', final_score)

In [70]:
# data = pd.read_parquet("Data/Training/Amex_Campus_Challenge_Train.parquet")
traindata = pd.read_csv("Data/Training/bilkulfirstaayenge.csv")

In [8]:
randomlist = random.sample(list(data.customer.unique()),int(0.1*len(data.customer.unique())))

In [8]:
testdata = data[data['customer'].isin(randomlist)]

In [71]:
#select rows whose column value is not in a list
# traindata = data[~data['customer'].isin(randomlist)]
traindata = data

NameError: name 'data' is not defined

In [72]:
del data

NameError: name 'data' is not defined

In [73]:
def confusion_matrix_2col(df, col1, col2):
    return df.groupby([col1, col2]).size().unstack(fill_value=0)

confusion_matrix_2col(traindata, 'ind_recommended', 'activation')

activation,0,1
ind_recommended,,
0,292879,60173
1,80162,9843


In [74]:
tdata = traindata

In [75]:
# tdata = traindata.drop(traindata[(traindata['activation'] == 0) & (traindata['ind_recommended'] == 0)].sample(10572736).index)
# tdata = tdata.drop(tdata[(tdata['activation'] == 0) & (tdata['ind_recommended'] == 1)].sample(1477226).index)
# tdata = traindata.drop(traindata[(traindata['activation'] == 0) & (traindata['ind_recommended'] == 0)].sample(212879).index)


tdata = traindata.drop(traindata[(traindata['activation'] == 0) & (traindata['ind_recommended'] == 0)].sample(282879).index)
tdata = tdata.drop(tdata[(tdata['activation'] == 0) & (tdata['ind_recommended'] == 1)].sample(70162).index)
tdata = traindata.drop(traindata[(traindata['activation'] == 1) & (traindata['ind_recommended'] == 0)].sample(50173).index)


In [76]:
def scoring(datax):
    datax['score'] = datax['activation']
    datax.loc[(datax['activation'] == 1) & (datax['ind_recommended'] == 1), 'score'] = 100
    datax.loc[(datax['activation'] == 0) & (datax['ind_recommended'] == 0), 'score'] = 50
    datax.loc[(datax['activation'] == 0) & (datax['ind_recommended'] == 1), 'score'] = -50
    datax.loc[(datax['activation'] == 1) & (datax['ind_recommended'] == 0), 'score'] = -100
    return datax


In [19]:
name = 'allapproachcompilation'
# name = 'AAC_newdata'

In [16]:
rec = joblib.load('submissions/'+name+'/models/recact.joblib')
nrec = joblib.load('submissions/'+name+'/models/nrecact.joblib')
uplift = joblib.load('submissions/'+name+'/models/upliftold.joblib')
# jarvis = joblib.load('submissions/'+name+'/models/rec.joblib')
# act = joblib.load('submissions/'+name+'/models/act.joblib')

In [17]:
name = 'sidmethodwithform1'

In [18]:
jarvis = joblib.load('submissions/'+name+'/models/rec.joblib')
act = joblib.load('submissions/'+name+'/models/act.joblib')

In [77]:
ttdata = tdata.copy()

In [78]:
tdata['recact'] = rec.predict_proba(ttdata.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]
tdata['nrecact'] = nrec.predict_proba(ttdata.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]
tdata['uplift'] = uplift.predict(ttdata.drop(['activation','ind_recommended'],axis=1))
tdata['jarvis'] = jarvis.predict_proba(ttdata.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]
tdata['act'] = act.predict_proba(ttdata.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]


In [79]:
tdata = scoring(tdata)

In [80]:
# lists = ['recact','nrecact','uplift','jarvis','act']
# lists = traindata.columns.drop(['customer','merchant','activation','ind_recommended'])
lists = tdata.columns.drop(['customer','merchant','activation','ind_recommended','score'])

In [81]:
X_train, X_test, y_train, y_test = train_test_split(tdata[lists], tdata['score'], test_size=0.1, random_state=1)
ranker = cbt.CatBoostRegressor()

In [82]:
ranker.fit(X_train, y_train)
y_pred = ranker.predict(X_test)
print("rmse : ",mean_squared_error(y_test, y_pred)**0.5)

Learning rate set to 0.103477
0:	learn: 46.0098852	total: 126ms	remaining: 2m 6s
1:	learn: 45.7622623	total: 164ms	remaining: 1m 21s
2:	learn: 45.5598631	total: 192ms	remaining: 1m 3s
3:	learn: 45.3766379	total: 216ms	remaining: 53.8s
4:	learn: 45.2323953	total: 244ms	remaining: 48.5s
5:	learn: 45.1077941	total: 271ms	remaining: 44.9s
6:	learn: 45.0072859	total: 298ms	remaining: 42.2s
7:	learn: 44.9227866	total: 324ms	remaining: 40.1s
8:	learn: 44.8563103	total: 347ms	remaining: 38.2s
9:	learn: 44.7951015	total: 373ms	remaining: 36.9s
10:	learn: 44.7383658	total: 394ms	remaining: 35.5s
11:	learn: 44.6917991	total: 418ms	remaining: 34.4s
12:	learn: 44.6488390	total: 437ms	remaining: 33.2s
13:	learn: 44.6059990	total: 458ms	remaining: 32.2s
14:	learn: 44.5763075	total: 484ms	remaining: 31.8s
15:	learn: 44.5451497	total: 507ms	remaining: 31.2s
16:	learn: 44.5277913	total: 528ms	remaining: 30.5s
17:	learn: 44.5008224	total: 550ms	remaining: 30s
18:	learn: 44.4802407	total: 571ms	remaining:

In [83]:
newlabels = ['recact','nrecact','uplift','jarvis','act']

In [84]:
for l in newlabels:
    try:
        testdata.drop(l,axis=1,inplace=True)
    except:
        pass

In [85]:
try:
    testdata.drop(['predicted_score'],axis=1,inplace=True)
except:
    pass



In [27]:
testdata = pd.read_csv("Data/Evaluation/part2.csv")

In [28]:
testdata

,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,customer_merchant_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,218.0,243.0,NaN,509.755,NaN,NaN,NaN,0.164692,574819,1087034
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.539078,574819,1328040
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8369.0,8960.0,NaN,1096.775,NaN,NaN,NaN,0.524643,574819,1502936
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13503.0,8277.0,NaN,669.960,NaN,NaN,NaN,0.460987,574819,1529762
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,917.0,919.0,NaN,114.495,NaN,NaN,NaN,0.759766,574819,1685993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4108129,NaN,631.800000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,44184.0,40128.0,0.068966,29.630,0.924042,0.500000,182.0,17.082029,581658,1685038
4108130,NaN,49.490000,NaN,NaN,NaN,NaN,NaN,2.0,9.0,NaN,...,NaN,NaN,0.357143,NaN,15.819092,9.333333,536.0,25.099151,559729,1741912
4108131,NaN,494.520000,NaN,NaN,NaN,NaN,NaN,7.0,26.0,NaN,...,NaN,NaN,0.235294,NaN,40.522384,0.000000,62.0,8.815975,594817,1414695
4108132,NaN,78.063171,62.619464,3.0,3506.69,56.0,54.0,13.0,63.0,NaN,...,10777.0,17744.0,0.054054,21.965,96.337732,6.833333,49.0,16.690912,597078,1414861


In [86]:
test = testdata.copy()

# testdata['recact'] = rec.predict_proba(test.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]
# testdata['nrecact'] = nrec.predict_proba(test.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]
# testdata['uplift'] = uplift.predict(test.drop(['activation','ind_recommended'],axis=1))
# testdata['jarvis'] = jarvis.predict_proba(test.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]
# testdata['act'] = act.predict_proba(test.drop(['customer','merchant','activation','ind_recommended'],axis=1))[:,1]

testdata['recact'] = rec.predict_proba(test.drop(['customer','merchant'],axis=1))[:,1]
testdata['nrecact'] = nrec.predict_proba(test.drop(['customer','merchant'],axis=1))[:,1]
testdata['uplift'] = uplift.predict(test)
testdata['jarvis'] = jarvis.predict_proba(test.drop(['customer','merchant'],axis=1))[:,1]
testdata['act'] = act.predict_proba(test.drop(['customer','merchant'],axis=1))[:,1]

In [88]:
# with newdata
# testdata['predicted_score'] = ranker.predict(testdata[lists]) submission&submissionsmol
# testdata['predicted_score'] = (testdata.jarvis-0.6)*testdata.act #formula 112   score = 0.00135607
# testdata['predicted_score'] = (testdata.jarvis-0.5)*testdata.act #formula 113   score = 0.00056704
# testdata['predicted_score'] = (testdata.jarvis-0.7)*testdata.act #formula 114   score = 0.00129071
# testdata['predicted_score'] = testdata.recact - testdata.nrecact #formula 201   score = -0.00010778
# testdata['predicted_score'] = testdata.uplift #formula 0   score = 0.00079199
# testdata['predicted_score'] = ranker.predict(testdata[lists]) #withall10kquantities formula all10k w models of ogdata score = 0.00046158


# with ogdata
# testdata['predicted_score'] = ranker.predict(testdata[lists]) #submission&submissionsmol score = 0.00077203
# testdata['predicted_score'] = (testdata.jarvis-0.6)*testdata.act #formula 112   score = 0.00132045
# testdata['predicted_score'] = (testdata.jarvis-0.5)*testdata.act #formula 113   score = 0.00074586
# testdata['predicted_score'] = (testdata.jarvis-0.7)*testdata.act #formula 114   score = 0.00141893
# testdata['predicted_score'] = testdata.recact - testdata.nrecact #formula 201   score = -7.098e-05
# testdata['predicted_score'] = testdata.uplift #formula 0   score = 0.00093185
# testdata['predicted_score'] =  testdata.nrecact - testdata.recact #formula 202   score = -0.00066719
# testdata['predicted_score'] =  testdata.act #formula 301   score = -0.00110613
# testdata['predicted_score'] =  -testdata.act #formula 301   score = -4.846e-05
# testdata['predicted_score'] = (testdata.jarvis-0.8)*testdata.act #formula 115   score = 0.001501
# testdata['predicted_score'] = (testdata.jarvis-0.9)*testdata.act #formula 116   score = 0.00096976
# testdata['predicted_score'] = -0.00054838 #formula 001   score = 0.00096976
# testdata['predicted_score'] = np.round(testdata.recact - testdata.nrecact) #formula 202   score = -7.098e-05
# testdata['predicted_score'] = np.sign(testdata.jarvis-0.8)*testdata.act #formula 117   score = 0.001595
# testdata['predicted_score'] = np.sign(testdata.jarvis-0.8)*np.round(testdata.act) #formula 118   score = 0.00072836

In [89]:
testdata[['customer','merchant','predicted_score']].to_csv('submissions/'+name+'/submissionall10k.csv',index=False)

In [ ]:
#limit of int32 is 2147483647
#limit of float32 is 3.4028235e+38 

In [ ]:
evalu(testdata[['customer','predicted_score','merchant']],testdata[['customer','ind_recommended','activation','merchant']])

/var/folders/yy/95rx0nm12jqc32p5svcxxv7h0000gn/T/ipykernel_8550/245156590.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['predicted_score'] = df_input['predicted_score'].round(10)


Input Files are Correct
   ind_recommended  avg_30d_act
0                0     0.000443
1                1     0.010317
Incremental Activation Rate for Top 10 ranked Merchants(dataset level):  0.0098742


In [ ]:
# testdata[['customer','predicted_score','merchant']].to_csv('d_pred.csv',index=False)

In [38]:
name='AAC_newdata'

In [39]:
import os
os.makedirs('submissions/'+name+'/models', exist_ok=True)

In [40]:
joblib.dump(rec, 'submissions/'+name+'/models/submissionreg.joblib')

['submissions/AAC_newdata/models/submissionreg.joblib']

In [52]:
testdata.predicted_score.mean()

-4.34638174207572

In [54]:
testdata

,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,customer_merchant_02,...,customer_profile_04,distance_05,customer,merchant,recact,nrecact,uplift,jarvis,act,predicted_score
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.164692,574819,1087034,0.027469,0.026725,-0.213404,0.340783,0.078015,8.848197
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.539078,574819,1328040,0.115168,0.194309,-0.377131,0.332479,0.330906,-11.353085
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.524643,574819,1502936,0.077625,0.176436,-0.183363,0.359608,0.339773,-20.177387
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.460987,574819,1529762,0.180017,0.166665,-0.381797,0.370461,0.244494,2.499434
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.759766,574819,1685993,0.100866,0.242101,-0.391488,0.382274,0.261398,-21.980983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4108129,NaN,631.800000,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,...,182.0,17.082029,581658,1685038,0.294189,0.511328,-0.445308,0.312888,0.507988,-58.017103
4108130,NaN,49.490000,NaN,NaN,NaN,NaN,NaN,2.0,9.0,NaN,...,536.0,25.099151,559729,1741912,0.007494,0.002946,-0.020119,0.426632,0.004938,3.848597
4108131,NaN,494.520000,NaN,NaN,NaN,NaN,NaN,7.0,26.0,NaN,...,62.0,8.815975,594817,1414695,0.074319,0.047667,-0.237616,0.230103,0.047549,23.246670
4108132,NaN,78.063171,62.619464,3.0,3506.69,56.0,54.0,13.0,63.0,NaN,...,49.0,16.690912,597078,1414861,0.032011,0.025607,0.184521,0.607418,0.024754,-10.592619


In [53]:
name

'AAC_newdata'